In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import GaussianNB

Importando Bibliotecas necessárias

Agora,Importar CSV

In [2]:
base = pd.read_csv('https://raw.githubusercontent.com/Gustavolorenzz/IA/main/Bases/student-por.csv', sep=';')
colunas_desejadas = ['absences','failures', 'higher','G1','G2','G3']

base = base[colunas_desejadas]
base

,absences,failures,higher,G1,G2,G3
0,4,0,yes,0,11,11
1,2,0,yes,9,11,11
2,6,0,yes,12,13,12
3,0,0,yes,14,14,14
4,0,0,yes,11,13,13
...,...,...,...,...,...,...
644,4,1,yes,10,11,10
645,4,0,yes,15,15,16
646,6,0,yes,11,12,9
647,6,0,yes,10,10,10


Mapeamento de Qualificação

In [3]:
def mapear_qualificacao(nota):
    if nota >= 15.5:
        return 'Excellent'
    elif nota >= 9.5:
        return 'Sufficient'
    else:
        return 'Poor'

# Aplicar a função às colunas G1, G2 e G3
base['G1'] = base['G1'].apply(mapear_qualificacao)
base['G2'] = base['G2'].apply(mapear_qualificacao)
base['G3'] = base['G3'].apply(mapear_qualificacao)

In [7]:
base

,absences,failures,higher,G1,G2,G3
0,Low level of absences,No fails,yes,Poor,Sufficient,Sufficient
1,Low level of absences,No fails,yes,Poor,Sufficient,Sufficient
2,Medium level of absences,No fails,yes,Sufficient,Sufficient,Sufficient
3,Low level of absences,No fails,yes,Sufficient,Sufficient,Sufficient
4,Low level of absences,No fails,yes,Sufficient,Sufficient,Sufficient
...,...,...,...,...,...,...
644,Low level of absences,Has failed before,yes,Sufficient,Sufficient,Sufficient
645,Low level of absences,No fails,yes,Sufficient,Sufficient,Excellent
646,Medium level of absences,No fails,yes,Sufficient,Sufficient,Poor
647,Medium level of absences,No fails,yes,Sufficient,Sufficient,Sufficient


In [5]:
def mapear_faltas(absences):
  if absences >= 10:
    return 'High level of absences'
  elif absences >= 5:
    return 'Medium level of absences'
  else:
    return 'Low level of absences'


#Aplicar função à coluna absences
base['absences'] = base['absences'].apply(mapear_faltas)

In [6]:
def mapear_bombas(failures):
  if(failures > 0):
    return 'Has failed before'
  else:
    return 'No fails'

#Aplicar função à coluna failures
base['failures'] = base['failures'].apply(mapear_bombas)

In [8]:
label_encoder_higher = LabelEncoder()
label_encoder_g1 = LabelEncoder()
label_encoder_g2 = LabelEncoder()
label_encoder_absences = LabelEncoder()
label_encoder_fails = LabelEncoder()

In [9]:
# Aplique o label encoder às colunas desejadas
base['higher'] = label_encoder_higher.fit_transform(base['higher'])
base['G1'] = label_encoder_g1.fit_transform(base['G1'])
base['G2'] = label_encoder_g2.fit_transform(base['G2'])
base['absences'] = label_encoder_absences.fit_transform(base['absences'])
base['failures'] = label_encoder_fails.fit_transform(base['failures'])

In [10]:
base

,absences,failures,higher,G1,G2,G3
0,1,1,1,1,2,Sufficient
1,1,1,1,1,2,Sufficient
2,2,1,1,2,2,Sufficient
3,1,1,1,2,2,Sufficient
4,1,1,1,2,2,Sufficient
...,...,...,...,...,...,...
644,1,0,1,2,2,Sufficient
645,1,1,1,2,2,Excellent
646,2,1,1,2,2,Poor
647,2,1,1,2,2,Sufficient


In [11]:
# Separe seus recursos (X) e rótulos (y)
y = base['G3']
X = base.drop('G3', axis=1)

In [12]:
# Crie uma instância do RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [13]:
# Use value_counts() para contar os valores únicos na série
contagem_classes = y_resampled.value_counts()

# A contagem das classes será armazenada em 'contagem_classes'
# Você pode acessar o número de 0s e 1s da seguinte forma:
numero_de_0s = contagem_classes.get(0, 0)  # O segundo argumento é o valor padrão se 0 não estiver presente
numero_de_1s = contagem_classes.get(1, 0)  # O segundo argumento é o valor padrão se 1 não estiver presente
numero_de_2s = contagem_classes.get(2, 0)  # O segundo argumento é o valor padrão se 2 não estiver presente

# Imprima os resultados
print(f"Número de 0s na série: {numero_de_0s}")
print(f"Número de 1s na série: {numero_de_1s}")
print(f"Número de 2s na série: {numero_de_2s}")

Número de 0s na série: 467
Número de 1s na série: 467
Número de 2s na série: 467


In [14]:
label_encoder_teste = LabelEncoder()
y_encoded = label_encoder_teste.fit_transform(y_resampled)

In [15]:
#Inicializar naive bayes
gnb = GaussianNB()
gnb.fit(X_resampled,y_encoded)

GaussianNB()

Agora,vetores de teste para achar as possibilidades de cada combinação de entradas

In [16]:
vetor_HighAb_Fail_HighN = [
                           [0,0,0,0,0],
                           [0,0,0,0,1],
                           [0,0,0,0,2],
                           [0,0,0,1,0],
                           [0,0,0,1,1],
                           [0,0,0,1,2],
                           [0,0,0,2,0],
                           [0,0,0,2,1],
                           [0,0,0,2,2]
                           ]
test1 = pd.DataFrame(data = vetor_HighAb_Fail_HighN,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test1)

array([[0.00000000e+00, 1.00000000e+00, 2.31202702e-10],
       [0.00000000e+00, 9.99999971e-01, 2.92958824e-08],
       [0.00000000e+00, 9.99952576e-01, 4.74242708e-05],
       [0.00000000e+00, 9.99999993e-01, 6.85087115e-09],
       [0.00000000e+00, 9.99999132e-01, 8.68078733e-07],
       [0.00000000e+00, 9.98596656e-01, 1.40334439e-03],
       [0.00000000e+00, 9.99995979e-01, 4.02052386e-06],
       [0.00000000e+00, 9.99490813e-01, 5.09186509e-04],
       [0.00000000e+00, 5.48025741e-01, 4.51974259e-01]])

In [17]:
vetor_HighAb_Fail_HighY = [
                           [0,0,1,0,0],
                           [0,0,1,0,1],
                           [0,0,1,0,2],
                           [0,0,1,1,0],
                           [0,0,1,1,1],
                           [0,0,1,1,2],
                           [0,0,1,2,0],
                           [0,0,1,2,1],
                           [0,0,1,2,2]
                           ]
test2 = pd.DataFrame(data = vetor_HighAb_Fail_HighY,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test2)

array([[3.23616717e-08, 9.99999923e-01, 4.43737698e-08],
       [5.25758632e-11, 9.99994377e-01, 5.62260545e-06],
       [1.31105326e-09, 9.90979730e-01, 9.02026888e-03],
       [1.02495742e-10, 9.99998685e-01, 1.31485749e-06],
       [1.66491443e-13, 9.99833421e-01, 1.66579133e-04],
       [3.30007791e-12, 7.87577348e-01, 2.12422652e-01],
       [2.90415326e-09, 9.99228947e-01, 7.71050549e-04],
       [4.30129006e-12, 9.10932786e-01, 8.90672141e-02],
       [7.45929583e-13, 6.27796260e-03, 9.93722037e-01]])

In [18]:
vetor_HighAb_NoFail_HighN = [
                           [0,1,0,0,0],
                           [0,1,0,0,1],
                           [0,1,0,0,2],
                           [0,1,0,1,0],
                           [0,1,0,1,1],
                           [0,1,0,1,2],
                           [0,1,0,2,0],
                           [0,1,0,2,1],
                           [0,1,0,2,2]
                           ]
test3 = pd.DataFrame(data = vetor_HighAb_NoFail_HighN,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test3)

array([[0.00000000e+00, 9.99999985e-01, 1.50276771e-08],
       [0.00000000e+00, 9.99998096e-01, 1.90416564e-06],
       [0.00000000e+00, 9.96926852e-01, 3.07314822e-03],
       [0.00000000e+00, 9.99999555e-01, 4.45291662e-07],
       [0.00000000e+00, 9.99943580e-01, 5.64201120e-05],
       [0.00000000e+00, 9.16302589e-01, 8.36974108e-02],
       [0.00000000e+00, 9.99738742e-01, 2.61258162e-04],
       [0.00000000e+00, 9.67948434e-01, 3.20515663e-02],
       [0.00000000e+00, 1.83130765e-02, 9.81686924e-01]])

In [19]:
vetor_HighAb_NoFail_HighY = [
                           [0,1,1,0,0],
                           [0,1,1,0,1],
                           [0,1,1,0,2],
                           [0,1,1,1,0],
                           [0,1,1,1,1],
                           [0,1,1,1,2],
                           [0,1,1,2,0],
                           [0,1,1,2,1],
                           [0,1,1,2,2]
                           ]
test4 = pd.DataFrame(data = vetor_HighAb_NoFail_HighY,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test4)

array([[9.99999999e-01, 6.03562000e-10, 1.74079329e-15],
       [9.99999628e-01, 3.71504309e-07, 1.35769701e-10],
       [9.99999977e-01, 1.47637709e-08, 8.73475975e-09],
       [9.99999809e-01, 1.90566428e-07, 1.62863815e-11],
       [9.99881446e-01, 1.17283553e-04, 1.27007397e-06],
       [9.99913626e-01, 4.66105665e-06, 8.17129603e-05],
       [9.99999993e-01, 6.72044982e-09, 3.37066151e-10],
       [9.99969576e-01, 4.13644506e-06, 2.62880123e-05],
       [9.98311399e-01, 1.64111794e-07, 1.68843733e-03]])

In [20]:
vetor_LowAb_Fail_HighN = [
                           [1,0,0,0,0],
                           [1,0,0,0,1],
                           [1,0,0,0,2],
                           [1,0,0,1,0],
                           [1,0,0,1,1],
                           [1,0,0,1,2],
                           [1,0,0,2,0],
                           [1,0,0,2,1],
                           [1,0,0,2,2]
                           ]
test5 = pd.DataFrame(data = vetor_LowAb_Fail_HighN,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test5)

array([[0.00000000e+00, 1.00000000e+00, 3.89821450e-10],
       [0.00000000e+00, 9.99999951e-01, 4.93945919e-08],
       [0.00000000e+00, 9.99920042e-01, 7.99575279e-05],
       [0.00000000e+00, 9.99999988e-01, 1.15509745e-08],
       [0.00000000e+00, 9.99998536e-01, 1.46363129e-06],
       [0.00000000e+00, 9.97636154e-01, 2.36384590e-03],
       [0.00000000e+00, 9.99993221e-01, 6.77882266e-06],
       [0.00000000e+00, 9.99141781e-01, 8.58218808e-04],
       [0.00000000e+00, 4.18314360e-01, 5.81685640e-01]])

In [21]:
vetor_LowAb_Fail_HighY = [
                           [1,0,1,0,0],
                           [1,0,1,0,1],
                           [1,0,1,0,2],
                           [1,0,1,1,0],
                           [1,0,1,1,1],
                           [1,0,1,1,2],
                           [1,0,1,2,0],
                           [1,0,1,2,1],
                           [1,0,1,2,2]
                           ]
test6 = pd.DataFrame(data = vetor_LowAb_Fail_HighY,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test6)

array([[9.47091601e-08, 9.99999830e-01, 7.48167972e-08],
       [1.53867116e-10, 9.99990520e-01, 9.48000923e-06],
       [3.81330978e-09, 9.84884828e-01, 1.51151683e-02],
       [2.99962186e-10, 9.99997783e-01, 2.21692561e-06],
       [4.87195608e-13, 9.99719170e-01, 2.80830190e-04],
       [8.42948947e-12, 6.87399510e-01, 3.12600490e-01],
       [8.49475645e-09, 9.98700642e-01, 1.29934955e-03],
       [1.18631846e-11, 8.58475322e-01, 1.41524678e-01],
       [1.29806641e-12, 3.73298914e-03, 9.96267011e-01]])

In [22]:
vetor_LowAb_NoFail_HighN = [
                           [1,1,0,0,0],
                           [1,1,0,0,1],
                           [1,1,0,0,2],
                           [1,1,0,1,0],
                           [1,1,0,1,1],
                           [1,1,0,1,2],
                           [1,1,0,2,0],
                           [1,1,0,2,1],
                           [1,1,0,2,2]
                           ]
test7 = pd.DataFrame(data = vetor_LowAb_NoFail_HighN,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test7)

array([[0.00000000e+00, 9.99999975e-01, 2.53375533e-08],
       [0.00000000e+00, 9.99996789e-01, 3.21053185e-06],
       [0.00000000e+00, 9.94829392e-01, 5.17060840e-03],
       [0.00000000e+00, 9.99999249e-01, 7.50787887e-07],
       [0.00000000e+00, 9.99904876e-01, 9.51239685e-05],
       [0.00000000e+00, 8.66544412e-01, 1.33455588e-01],
       [0.00000000e+00, 9.99559582e-01, 4.40417795e-04],
       [0.00000000e+00, 9.47121912e-01, 5.28780883e-02],
       [0.00000000e+00, 1.09430099e-02, 9.89056990e-01]])

In [23]:
vetor_LowAb_NoFail_HighY = [
                           [1,1,1,0,0],
                           [1,1,1,0,1],
                           [1,1,1,0,2],
                           [1,1,1,1,0],
                           [1,1,1,1,1],
                           [1,1,1,1,2],
                           [1,1,1,2,0],
                           [1,1,1,2,1],
                           [1,1,1,2,2]
                           ]
test8 = pd.DataFrame(data = vetor_LowAb_NoFail_HighY,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test8)

array([[1.00000000e+00, 2.06234260e-10, 1.00290305e-15],
       [9.99999873e-01, 1.26941283e-07, 7.82194424e-11],
       [9.99999990e-01, 5.04471024e-09, 5.03225592e-09],
       [9.99999935e-01, 6.51156485e-08, 9.38288530e-12],
       [9.99959190e-01, 4.00783471e-05, 7.31769881e-07],
       [9.99951329e-01, 1.59272090e-06, 4.70781154e-05],
       [9.99999998e-01, 2.29634569e-09, 1.94190014e-10],
       [9.99983441e-01, 1.41342316e-06, 1.51452177e-05],
       [9.99026507e-01, 5.61163866e-08, 9.73436610e-04]])

In [24]:
vetor_MedAb_Fail_HighN = [
                           [2,0,0,0,0],
                           [2,0,0,0,1],
                           [2,0,0,0,2],
                           [2,0,0,1,0],
                           [2,0,0,1,1],
                           [2,0,0,1,2],
                           [2,0,0,2,0],
                           [2,0,0,2,1],
                           [2,0,0,2,2]
                           ]
test9 = pd.DataFrame(data = vetor_MedAb_Fail_HighN,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test9)

array([[0.00000000e+00, 1.00000000e+00, 2.95750735e-10],
       [0.00000000e+00, 9.99999963e-01, 3.74748159e-08],
       [0.00000000e+00, 9.99939336e-01, 6.06635525e-05],
       [0.00000000e+00, 9.99999991e-01, 8.76352290e-09],
       [0.00000000e+00, 9.99998890e-01, 1.11043192e-06],
       [0.00000000e+00, 9.98205568e-01, 1.79443227e-03],
       [0.00000000e+00, 9.99994857e-01, 5.14298294e-06],
       [0.00000000e+00, 9.99348749e-01, 6.51250518e-04],
       [0.00000000e+00, 4.86621910e-01, 5.13378090e-01]])

In [25]:
vetor_MedAb_Fail_HighY = [
                           [2,0,1,0,0],
                           [2,0,1,0,1],
                           [2,0,1,0,2],
                           [2,0,1,1,0],
                           [2,0,1,1,1],
                           [2,0,1,1,2],
                           [2,0,1,2,0],
                           [2,0,1,2,1],
                           [2,0,1,2,2]
                           ]
test10 = pd.DataFrame(data = vetor_MedAb_Fail_HighY,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test10)

array([[4.91067931e-08, 9.99999894e-01, 5.67622028e-08],
       [7.97804292e-11, 9.99992808e-01, 7.19233412e-06],
       [1.98444313e-09, 9.88490402e-01, 1.15095964e-02],
       [1.55530763e-10, 9.99998318e-01, 1.68194369e-06],
       [2.52628509e-13, 9.99786925e-01, 2.13075407e-04],
       [4.72730726e-12, 7.43485008e-01, 2.56514992e-01],
       [4.40592085e-09, 9.99013893e-01, 9.86103043e-04],
       [6.36857451e-12, 8.88831010e-01, 1.11168990e-01],
       [8.86076074e-13, 4.91452178e-03, 9.95085478e-01]])

In [26]:
vetor_MedAb_NoFail_HighN = [
                           [2,1,0,0,0],
                           [2,1,0,0,1],
                           [2,1,0,0,2],
                           [2,1,0,1,0],
                           [2,1,0,1,1],
                           [2,1,0,1,2],
                           [2,1,0,2,0],
                           [2,1,0,2,1],
                           [2,1,0,2,2]
                           ]
test11 = pd.DataFrame(data = vetor_MedAb_NoFail_HighN,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test11)

array([[0.00000000e+00, 9.99999981e-01, 1.92231599e-08],
       [0.00000000e+00, 9.99997564e-01, 2.43577641e-06],
       [0.00000000e+00, 9.96072249e-01, 3.92775129e-03],
       [0.00000000e+00, 9.99999430e-01, 5.69609779e-07],
       [0.00000000e+00, 9.99927829e-01, 7.21705526e-05],
       [0.00000000e+00, 8.95380278e-01, 1.04619722e-01],
       [0.00000000e+00, 9.99665827e-01, 3.34172816e-04],
       [0.00000000e+00, 9.59363782e-01, 4.06362180e-02],
       [0.00000000e+00, 1.43736701e-02, 9.85626330e-01]])

In [27]:
vetor_MedAb_NoFail_HighY = [
                           [2,1,1,0,0],
                           [2,1,1,0,1],
                           [2,1,1,0,2],
                           [2,1,1,1,0],
                           [2,1,1,1,1],
                           [2,1,1,1,2],
                           [2,1,1,2,0],
                           [2,1,1,2,1],
                           [2,1,1,2,2]
                           ]
test12 = pd.DataFrame(data = vetor_MedAb_NoFail_HighY,columns = ['absences','failures','higher','G1','G2'])
gnb.predict_proba(test12)

array([[1.00000000e+00, 3.97750973e-10, 1.46747088e-15],
       [9.99999755e-01, 2.44823596e-07, 1.14452479e-10],
       [9.99999983e-01, 9.72941352e-09, 7.36331290e-09],
       [9.99999874e-01, 1.25584425e-07, 1.37292534e-11],
       [9.99921636e-01, 7.72936704e-05, 1.07070236e-06],
       [9.99928044e-01, 3.07170854e-06, 6.88841804e-05],
       [9.99999995e-01, 4.42881669e-09, 2.84143307e-10],
       [9.99975113e-01, 2.72595716e-06, 2.21606474e-05],
       [9.98576179e-01, 1.08179340e-07, 1.42371285e-03]])